# Stage 1 Merge and Clean data


In [1]:
# --- Bước 0: Khởi tạo và Tải Thư viện ---
import pandas as pd
import numpy as np
import os # Dùng để quản lý đường dẫn file
from IPython.display import display

# Cấu hình Pandas để hiển thị tất cả các cột
pd.set_option('display.max_columns', None)
print("Các thư viện đã sẵn sàng.")

Các thư viện đã sẵn sàng.


In [2]:
DATA_DIR = 'data/' # Đảm bảo thư mục 'data' tồn tại
print(f"Bắt đầu tải 9 tệp dữ liệu từ thư mục: {DATA_DIR}")

try:
    df_orders = pd.read_csv(os.path.join(DATA_DIR, 'olist_orders_dataset.csv'))
    df_items = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_items_dataset.csv'))
    df_products = pd.read_csv(os.path.join(DATA_DIR, 'olist_products_dataset.csv'))
    df_customers = pd.read_csv(os.path.join(DATA_DIR, 'olist_customers_dataset.csv'))
    df_reviews = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_reviews_dataset.csv'))
    df_payments = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_payments_dataset.csv'))
    df_sellers = pd.read_csv(os.path.join(DATA_DIR, 'olist_sellers_dataset.csv'))
    df_geo = pd.read_csv(os.path.join(DATA_DIR, 'olist_geolocation_dataset.csv'))
    df_translation = pd.read_csv(os.path.join(DATA_DIR, 'product_category_name_translation.csv'))

    print("✓ Tải 9 tệp dữ liệu chính thành công.")
    print(f"Orders: {df_orders.shape}, Items: {df_items.shape}, Payments: {df_payments.shape}, Geo: {df_geo.shape}")

except FileNotFoundError as e:
    print(f"LỖI: Không tìm thấy tệp. {e}")
    print("Vui lòng kiểm tra lại đường dẫn và tên tệp trong thư mục 'data'.")
    # Thoát nếu lỗi để không chạy tiếp
    exit()

Bắt đầu tải 9 tệp dữ liệu từ thư mục: data/
✓ Tải 9 tệp dữ liệu chính thành công.
Orders: (99441, 8), Items: (112650, 7), Payments: (103886, 5), Geo: (1000163, 5)


## Bước 2: (QUAN TRỌNG) Xử lý "Bẫy Hợp nhất" 💣 - Bảng `order_payments`

**Vấn đề:** Bảng `payments` có quan hệ 1-Nhiều (1 `order_id` có thể có nhiều hàng thanh toán, ví dụ: 1 voucher + 1 thẻ tín dụng). Nếu merge trực tiếp với `items` (cũng là 1-Nhiều), dữ liệu sẽ bị nhân lên sai lệch.

**Giải pháp:** Chúng ta phải **aggregate (gộp)** bảng `payments` theo `order_id` *trước khi* merge.

In [3]:
print("\n" + "="*50)
print("BƯỚC 2: TIỀN XỬ LÝ BẢNG 1-N (CHỐNG BÙNG NỔ DÒNG)")
print("="*50)

# ---------------------------------------------------------------------------
# Xử lý Bảng 'df_payments' (Quan hệ 1-Nhiều)
# ---------------------------------------------------------------------------
print(f"Bảng Payments ban đầu (thô): {df_payments.shape[0]} dòng")
print(f"Số order_id duy nhất trong Payments: {df_payments['order_id'].nunique()} dòng")
print("Bắt đầu gộp (aggregate) bảng Payments...")

df_payments_agg = df_payments.groupby('order_id').agg(
    # --- Đặc trưng Tài chính ---
    # Tính TỔNG giá trị thanh toán cho đơn hàng đó
    payment_value_total=('payment_value', 'sum'),

    # Tính TỔNG số kỳ trả góp (ví dụ: 1 lần thẻ 3 kỳ + 1 lần thẻ 3 kỳ = 6 kỳ)
    payment_installments_total=('payment_installments', 'sum'),

    # --- Đặc trưng Phân loại ---
    # Lấy loại thanh toán chính (thường là 'first' hoặc 'mode')
    payment_type_primary=('payment_type', 'first'),

    # Đếm xem đơn hàng này được thanh toán bằng bao nhiêu phương thức/lần
    payment_sequential_count=('payment_sequential', 'max')

).reset_index()

print(f"✓ Bảng Payments sau khi gộp (Aggregated): {df_payments_agg.shape[0]} dòng")
print("\nKiểm tra 5 dòng đầu của bảng payments_agg:")
print(df_payments_agg.head())
print("="*50)


BƯỚC 2: TIỀN XỬ LÝ BẢNG 1-N (CHỐNG BÙNG NỔ DÒNG)
Bảng Payments ban đầu (thô): 103886 dòng
Số order_id duy nhất trong Payments: 99440 dòng
Bắt đầu gộp (aggregate) bảng Payments...
✓ Bảng Payments sau khi gộp (Aggregated): 99440 dòng

Kiểm tra 5 dòng đầu của bảng payments_agg:
                           order_id  payment_value_total  \
0  00010242fe8c5a6d1ba2dd792cb16214                72.19   
1  00018f77f2f0320c557190d7a144bdd3               259.83   
2  000229ec398224ef6ca0657da4fc703e               216.87   
3  00024acbcdf0a6daa1e931b038114c75                25.78   
4  00042b26cf59d7ce69dfabb4e55b4fd9               218.04   

   payment_installments_total payment_type_primary  payment_sequential_count  
0                           2          credit_card                         1  
1                           3          credit_card                         1  
2                           5          credit_card                         1  
3                           2          credit_

In [4]:
print("Đang xử lý trùng lặp bảng Reviews...")

# Sắp xếp theo ngày tạo và giữ lại review cuối cùng (mới nhất) cho mỗi order_id
df_reviews_dedup = df_reviews.sort_values(
    'review_creation_date', ascending=False
).drop_duplicates(
    'order_id', keep='first'
).reset_index()

print(f"Bảng Reviews ban đầu: {df_reviews.shape[0]} dòng")
print(f"Bảng Reviews sau khi dedup: {df_reviews_dedup.shape[0]} dòng")

# Sau đó, ở Bước 3.3, bạn merge với df_reviews_dedup này
# df_master = df_master.merge(df_reviews_dedup, on='order_id', how='left')

Đang xử lý trùng lặp bảng Reviews...
Bảng Reviews ban đầu: 99224 dòng
Bảng Reviews sau khi dedup: 98673 dòng


In [5]:
print("\n" + "="*50)
print("BƯỚC 2.6: TIỀN XỬ LÝ BẢNG GEOLOCATION (AGGREGATE)")
print("="*50)
print("Đang gộp (Aggregate) bảng 'geolocation'...")

# Lấy tọa độ trung bình cho mỗi zip code
df_geo_map = df_geo.groupby('geolocation_zip_code_prefix').agg(
    geolocation_lat=('geolocation_lat', 'mean'),
    geolocation_lng=('geolocation_lng', 'mean')
).reset_index()

print(f"-> Đã gộp 'geolocation' từ {len(df_geo)} hàng xuống {len(df_geo_map)} hàng (zip codes duy nhất).")
print("✓ Đã tạo 'df_geo_map' sẵn sàng để merge.")
print("="*50)


BƯỚC 2.6: TIỀN XỬ LÝ BẢNG GEOLOCATION (AGGREGATE)
Đang gộp (Aggregate) bảng 'geolocation'...
-> Đã gộp 'geolocation' từ 1000163 hàng xuống 19015 hàng (zip codes duy nhất).
✓ Đã tạo 'df_geo_map' sẵn sàng để merge.


## Bước 3: Thực thi Pipeline Hợp nhất Lõi (Core Merge Pipeline)

Chúng ta sẽ xây dựng Bảng Tổng thể (Master Table) theo Sơ đồ (Schema) đã vẽ.
Luồng (Flow): Bắt đầu từ `df_orders` và `merge` các bảng khác vào.

In [6]:
print("\n" + "="*50)
print("BƯỚC 3: THỰC THI PIPELINE HỢP NHẤT LÕI")
print("="*50)

# ---------------------------------------------------------------------------
# 3.1: Khởi tạo Master Table và Merge "Explode" (1-N)
# ---------------------------------------------------------------------------

# Bắt đầu "xương sống" (backbone) từ df_orders
df_master = df_orders.copy()
print(f"Shape ban đầu (Orders): {df_master.shape}")

# Merge df_items. Đây là bước "làm nổ" (explode) số dòng.
# Mỗi dòng giờ đây đại diện cho một (order, item)
# Chúng ta dùng 'how=left' để giữ các đơn hàng (nếu có) mà không có items
df_master = df_master.merge(df_items, on='order_id', how='left')
print(f"Shape sau khi merge df_items (exploded): {df_master.shape}")

# ---------------------------------------------------------------------------
# 3.2: Merge các Bảng 1-1 (tương ứng với Item)
# ---------------------------------------------------------------------------

# Thêm thông tin sản phẩm (product)
df_master = df_master.merge(df_products, on='product_id', how='left')
print(f"Shape sau khi merge df_products: {df_master.shape}")

# Thêm thông tin người bán (seller)
df_master = df_master.merge(df_sellers, on='seller_id', how='left')
print(f"Shape sau khi merge df_sellers: {df_master.shape}")

# Thêm bản dịch tên danh mục
df_master = df_master.merge(df_translation, on='product_category_name', how='left')
print(f"Shape sau khi merge df_translation: {df_master.shape}")

# ---------------------------------------------------------------------------
# 3.3: Merge các Bảng 1-1 (tương ứng với Order)
# ---------------------------------------------------------------------------

# Thêm thông tin khách hàng (customer)
df_master = df_master.merge(df_customers, on='customer_id', how='left')
print(f"Shape sau khi merge df_customers: {df_master.shape}")

# Thêm thông tin thanh toán (đã gộp)
# Sử dụng `df_payments_agg` (đã xử lý ở Bước 2)
df_master = df_master.merge(df_payments_agg, on='order_id', how='left')
print(f"Shape sau khi merge df_payments_agg: {df_master.shape}")

# Thêm thông tin review (đã gộp/dedup)
# (Giả sử bạn đã tạo df_reviews_dedup)
df_master = df_master.merge(df_reviews_dedup, on='order_id', how='left')
print(f"Shape sau khi merge df_reviews_dedup: {df_master.shape}")

# ---------------------------------------------------------------------------
# 3.4: Merge Bảng GEO (Phức tạp, 2 lần)
# ---------------------------------------------------------------------------
# (Giả sử bạn đã tạo df_geo_map)

# 1. Merge tọa độ Khách hàng
df_master = df_master.merge(
    df_geo_map,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)
# Đổi tên ngay để tránh xung đột ở lần merge tiếp theo
df_master = df_master.rename(columns={
    'geolocation_lat': 'customer_lat',
    'geolocation_lng': 'customer_lng'
})
# Dọn dẹp cột key
if 'geolocation_zip_code_prefix' in df_master.columns:
    df_master = df_master.drop(columns=['geolocation_zip_code_prefix'])

print(f"Shape sau khi merge Geo (Customer): {df_master.shape}")

# 2. Merge tọa độ Người bán
df_master = df_master.merge(
    df_geo_map,
    left_on='seller_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left',
    suffixes=('_customer', '_seller') # Thêm hậu tố cho cột key (nếu còn)
)
df_master = df_master.rename(columns={
    'geolocation_lat': 'seller_lat',
    'geolocation_lng': 'seller_lng'
})
# Dọn dẹp các cột key thừa
if 'geolocation_zip_code_prefix' in df_master.columns:
    df_master = df_master.drop(columns=['geolocation_zip_code_prefix'])

print(f"Shape sau khi merge Geo (Seller): {df_master.shape}")

# ---------------------------------------------------------------------------
# BƯỚC 4: LƯU KẾT QUẢ THÔ
# ---------------------------------------------------------------------------
OUTPUT_FILE = 'olist_raw_merged.csv'
try:
    df_master.to_csv(OUTPUT_FILE, index=False)
    print("\n" + "="*50)
    print(f"✓✓✓ PIPELINE HỢP NHẤT HOÀN TẤT ✓✓✓")
    print(f"Shape cuối cùng của Master Table: {df_master.shape}")
    print(f"Đã lưu Master Table thô vào file: {OUTPUT_FILE}")
    print("\nKiểm tra 5 dòng đầu tiên (định dạng bảng):")

    display(df_master.head())
    # ---------------------------------

except Exception as e:
    print(f"\nLỖI khi lưu file: {e}")



BƯỚC 3: THỰC THI PIPELINE HỢP NHẤT LÕI
Shape ban đầu (Orders): (99441, 8)
Shape sau khi merge df_items (exploded): (113425, 14)
Shape sau khi merge df_products: (113425, 22)
Shape sau khi merge df_sellers: (113425, 25)
Shape sau khi merge df_translation: (113425, 26)
Shape sau khi merge df_customers: (113425, 30)
Shape sau khi merge df_payments_agg: (113425, 34)
Shape sau khi merge df_reviews_dedup: (113425, 41)
Shape sau khi merge Geo (Customer): (113425, 43)
Shape sau khi merge Geo (Seller): (113425, 45)

✓✓✓ PIPELINE HỢP NHẤT HOÀN TẤT ✓✓✓
Shape cuối cùng của Master Table: (113425, 45)
Đã lưu Master Table thô vào file: olist_raw_merged.csv

Kiểm tra 5 dòng đầu tiên (định dạng bảng):


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,product_category_name_english,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,payment_value_total,payment_installments_total,payment_type_primary,payment_sequential_count,index,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_lat,customer_lng,seller_lat,seller_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,housewares,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,38.71,3.0,credit_card,3.0,32790.0,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,-23.576983,-46.587161,-23.680729,-46.444238
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,perfumery,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,141.46,1.0,boleto,1.0,29158.0,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,-12.177924,-44.660711,-19.807681,-43.980427
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,14840.0,guariba,SP,auto,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,179.12,3.0,credit_card,1.0,4323.0,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58,-16.745150,-48.514783,-21.363502,-48.229601
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0,31842.0,belo horizonte,MG,pet_shop,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,72.20,1.0,credit_card,1.0,77520.0,359d03e676b3c069f62cadba8dd3f6e8,5.0,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58,-5.774190,-35.271143,-19.837682,-43.924053
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0,8752.0,mogi das cruzes,SP,stationery,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,28.62,1.0,credit_card,1.0,54136.0,e50934924e227544ba8246aeb3770dd4,5.0,NaN,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51,-23.676370,-46.514627,-23.543395,-46.262086


# Clean

In [7]:
# kiểm tra sơ bộ

print("\n" + "="*50)
print("CHECK ")
print("="*50)

df_master.info()
#check(missing value)
total_rows = len(df_master)
missing_summary = df_master.isna().sum().reset_index()
missing_summary.columns = ['column', 'num_missing']
missing_summary['pct_missing'] = (missing_summary['num_missing'] * 100.0 / total_rows)

# Chỉ hiển thị các cột CÓ giá trị thiếu
missing_summary = missing_summary[missing_summary['num_missing'] > 0].sort_values('pct_missing', ascending=False)

if missing_summary.empty:
    print("✓ Không có cột nào bị thiếu giá trị.")
else:
    print(f"✗ Tìm thấy {len(missing_summary)} cột có giá trị thiếu:")
    display(missing_summary)
#(chcek outlier)
numeric_stats = df_master.describe()
display(numeric_stats)


CHECK 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113425 entries, 0 to 113424
Data columns (total 45 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       113425 non-null  object 
 1   customer_id                    113425 non-null  object 
 2   order_status                   113425 non-null  object 
 3   order_purchase_timestamp       113425 non-null  object 
 4   order_approved_at              113264 non-null  object 
 5   order_delivered_carrier_date   111457 non-null  object 
 6   order_delivered_customer_date  110196 non-null  object 
 7   order_estimated_delivery_date  113425 non-null  object 
 8   order_item_id                  112650 non-null  float64
 9   product_id                     112650 non-null  object 
 10  seller_id                      112650 non-null  object 
 11  shipping_limit_date            112650 non-null  object 
 12  price                 

,column,num_missing,pct_missing
37,review_comment_title,99919,88.092572
38,review_comment_message,65507,57.753582
6,order_delivered_customer_date,3229,2.846815
25,product_category_name_english,2402,2.117699
14,product_category_name,2378,2.096540
16,product_description_lenght,2378,2.096540
17,product_photos_qty,2378,2.096540
15,product_name_lenght,2378,2.096540
5,order_delivered_carrier_date,1968,1.735067
43,seller_lat,1028,0.906326


,order_item_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,customer_zip_code_prefix,payment_value_total,payment_installments_total,payment_sequential_count,index,review_score,customer_lat,customer_lng,seller_lat,seller_lng
count,112650.000000,112650.000000,112650.000000,111047.000000,111047.000000,111047.000000,112632.000000,112632.000000,112632.000000,112632.000000,112650.000000,113425.000000,113422.000000,113422.000000,113422.000000,112464.000000,112464.000000,113119.000000,113119.000000,112397.000000,112397.000000
mean,1.197834,120.653739,19.990320,48.775978,787.867029,2.209713,2093.672047,30.153669,16.593766,22.996546,24439.170431,35102.472965,180.482857,3.067121,1.044956,49583.514058,4.017374,-21.236147,-46.203868,-22.797711,-47.235518
std,0.705124,183.633928,15.806405,10.025581,652.135608,1.721438,3751.596884,16.153449,13.443483,11.707268,27596.030909,29864.919733,273.548227,2.830879,0.386987,28660.238514,1.399652,5.567214,4.036390,2.697457,2.339961
min,1.000000,0.850000,0.000000,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000,1001.000000,1003.000000,0.000000,0.000000,1.000000,0.000000,1.000000,-33.689948,-72.668881,-32.079231,-67.813284
25%,1.000000,39.900000,13.080000,42.000000,348.000000,1.000000,300.000000,18.000000,8.000000,15.000000,6429.000000,11250.000000,65.680000,1.000000,1.000000,24765.750000,4.000000,-23.591282,-48.109940,-23.607473,-48.829744
50%,1.000000,74.990000,16.260000,52.000000,603.000000,1.000000,700.000000,25.000000,13.000000,20.000000,13568.000000,24320.000000,114.400000,2.000000,1.000000,49539.500000,5.000000,-22.930174,-46.634236,-23.425556,-46.743468
75%,1.000000,134.900000,21.150000,57.000000,987.000000,3.000000,1800.000000,38.000000,20.000000,30.000000,27930.000000,59020.000000,195.360000,4.000000,1.000000,74342.250000,5.000000,-20.179524,-43.644304,-21.757321,-46.518679
max,21.000000,6735.000000,409.680000,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000,99730.000000,99990.000000,13664.080000,29.000000,29.000000,99223.000000,5.000000,42.184003,-8.723762,-2.501242,-34.855616


In [8]:
# và chúng ta tạo 'df_master_clean' để làm sạch
df_master_clean = df_master.copy()

print("\n" + "="*50)
print("BẮT ĐẦU QUÁ TRÌNH LÀM SẠCH (CLEAN)")
print("="*50)

# --- 1. Dữ liệu Phân loại (Categorical): Dùng Giá trị Lính canh ---

# Reasoning: 'NaN' ở đây mang ý nghĩa nghiệp vụ là "không có review".
# Điền 0 giúp mô hình phân biệt được "chưa review" (0) với "review 1 sao" (1).
# Logic này lấy từ notebook 'Audit and Clean.ipynb' của bạn
df_master_clean['review_score'] = df_master_clean['review_score'].fillna(0)
df_master_clean['review_id'] = df_master_clean['review_id'].fillna('no_review')
print("✓ 1. Đã điền 'review_score' (NaN) = 0 và 'review_id' (NaN) = 'no_review'.")

print("✓ 1. Đã điền 'review_score' (NaN) = 0.")

# Reasoning: 'NaN' là một danh mục bị thiếu.
# Điền 'unknown' (không rõ) giúp mô hình học đây là 1 nhóm danh mục riêng.
df_master_clean['product_category_name'] = df_master_clean['product_category_name'].fillna('unknown')
df_master_clean['product_category_name_english'] = df_master_clean['product_category_name_english'].fillna('unknown')
print("✓ 2. Đã điền 'product_category_name' (NaN) = 'unknown'.")


# --- 2. Dữ liệu Địa lý (Geospatial): Dùng Điền khuyết theo Nhóm ---

# Reasoning: Điền 0 hoặc mean (trung bình) toàn cục là SAI (sẽ đặt vị trí ra giữa biển).
# Tối ưu: Dùng groupby().transform() để điền NaN bằng giá trị trung bình
# của chính zip_code_prefix (vùng) đó. Đây là kỹ thuật imputation chính xác nhất.
print("✓ 3. Đang điền khuyết (impute) tọa độ GEO (có thể mất vài giây)...")

# Điền cho Customer (Khách hàng)
geo_cols_cust = ['customer_lat', 'customer_lng']
zip_group_cust = 'customer_zip_code_prefix'
for col in geo_cols_cust:
    # Tính trung bình theo zip_code và map ngược lại
    mean_val_cust = df_master_clean.groupby(zip_group_cust)[col].transform('mean')
    df_master_clean[col] = df_master_clean[col].fillna(mean_val_cust)

# Điền cho Seller (Người bán)
geo_cols_seller = ['seller_lat', 'seller_lng']
zip_group_seller = 'seller_zip_code_prefix'
for col in geo_cols_seller:
    # Tính trung bình theo zip_code và map ngược lại
    mean_val_seller = df_master_clean.groupby(zip_group_seller)[col].transform('mean')
    df_master_clean[col] = df_master_clean[col].fillna(mean_val_seller)

# Điền nốt những zip_code hoàn toàn không có tọa độ (nếu có) bằng 0
df_master_clean[geo_cols_cust + geo_cols_seller] = df_master_clean[geo_cols_cust + geo_cols_seller].fillna(0)
print("   -> Đã điền tọa độ GEO (NaN) bằng trung bình của zip_code (hoặc 0).")


# --- 3. Dữ liệu Thời gian (Datetime): Không điền khuyết ---

# Reasoning: Chúng ta KHÔNG điền các cột datetime (như order_delivered_customer_date)
# bị NaT (Not a Time). Nó mang ý nghĩa "chưa giao hàng" hoặc "bị hủy".
# Việc xử lý sẽ được thực hiện ở Giai đoạn Tạo Đặc trưng (ví dụ: delivery_time = -999).
print("✓ 4. Giữ nguyên NaT cho các cột datetime (ý nghĩa nghiệp vụ 'chưa xảy ra').")


# --- 4. Dữ liệu Số (Numeric Outliers): Dùng Logic Nghiệp vụ ---

# Reasoning: price <= 0 là vô lý và sẽ gây lỗi chia cho 0 khi tạo 'freight_ratio'.
# Tối ưu: Chuyển các giá trị này thành NaN, sau đó điền bằng
# trung vị (median) của 'product_category' mà nó thuộc về.
df_master_clean.loc[df_master_clean['price'] <= 0, 'price'] = np.nan
median_price = df_master_clean.groupby('product_category_name')['price'].transform('median')
df_master_clean['price'] = df_master_clean['price'].fillna(median_price)
# Điền nốt nếu cả category cũng là NaN (hiếm)
df_master_clean['price'] = df_master_clean['price'].fillna(df_master_clean['price'].median())
print("✓ 5. Đã xử lý 'price' (<= 0) bằng median của category.")

# Reasoning: freight_value < 0 là vô lý.
# Tối ưu: Dùng .loc để gán trực tiếp = 0 (nhanh nhất).
df_master_clean.loc[df_master_clean['freight_value'] < 0, 'freight_value'] = 0
print("✓ 6. Đã xử lý 'freight_value' (< 0) = 0.")

print("\n" + "="*50)
print("✓✓✓ QUÁ TRÌNH LÀM SẠCH (CLEAN) HOÀN TẤT ✓✓✓")
print("="*50)
# Đây là file "sạch", đã merge, đã clean,
# nhưng CHƯA có các đặc trưng (features) như delivery_time_days
CLEAN_OUTPUT_FILE = 'olist_master_table_CLEAN_v1.csv'

print("\n" + "="*50)
print(f"BẮT ĐẦU LƯU BẢNG SẠCH RA FILE: {CLEAN_OUTPUT_FILE}")
print("="*50)

try:
    # Sử dụng df_master_clean (biến đã sạch từ Phần 5)
    df_master_clean.to_csv(CLEAN_OUTPUT_FILE, index=False)

    print(f"✓✓✓ THÀNH CÔNG! Đã lưu bảng sạch vào file: {CLEAN_OUTPUT_FILE}")
    print(f"Shape: {df_master_clean.shape}")
    print("\nNOTEBOOK 1 (MERGE & CLEAN) HOÀN TẤT.")

except Exception as e:
    print(f"\nLỖI KHI LƯU FILE: {e}")


BẮT ĐẦU QUÁ TRÌNH LÀM SẠCH (CLEAN)
✓ 1. Đã điền 'review_score' (NaN) = 0 và 'review_id' (NaN) = 'no_review'.
✓ 1. Đã điền 'review_score' (NaN) = 0.
✓ 2. Đã điền 'product_category_name' (NaN) = 'unknown'.
✓ 3. Đang điền khuyết (impute) tọa độ GEO (có thể mất vài giây)...
   -> Đã điền tọa độ GEO (NaN) bằng trung bình của zip_code (hoặc 0).
✓ 4. Giữ nguyên NaT cho các cột datetime (ý nghĩa nghiệp vụ 'chưa xảy ra').
✓ 5. Đã xử lý 'price' (<= 0) bằng median của category.
✓ 6. Đã xử lý 'freight_value' (< 0) = 0.

✓✓✓ QUÁ TRÌNH LÀM SẠCH (CLEAN) HOÀN TẤT ✓✓✓

BẮT ĐẦU LƯU BẢNG SẠCH RA FILE: olist_master_table_CLEAN_v1.csv
✓✓✓ THÀNH CÔNG! Đã lưu bảng sạch vào file: olist_master_table_CLEAN_v1.csv
Shape: (113425, 45)

NOTEBOOK 1 (MERGE & CLEAN) HOÀN TẤT.
